# RUL Prediction Video

In [32]:
import numpy as np
import torch
from pickle import load,dump
from datetime import date

# custom modules
from method import compute_eval_matrices
from watercan import GroupWaterCan
from visualization import make_rul_video 

In [33]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Get the data

In [34]:
dataset_name='uav_data'#'N-CMAPSS' #
threshold_name=''#'-0.1' #  '+0.1' #

# Load testing varblocks and EOL Causes

In [35]:
with open(f'./data/{dataset_name}/performs_info.pkl', 'rb') as file:
    performs_info=load(file)

with open(f'./data/{dataset_name}/times_test{threshold_name}.pkl', 'rb') as file:
    times = load(file)

with open(f'./data/{dataset_name}/times_test{threshold_name}_threshold.pkl', 'rb') as file:
    times_thresholds = load(file)

with open(f'./data/{dataset_name}/RUL_test{threshold_name}.pkl', 'rb') as file:
    rul= load(file)

with open(f'./data/{dataset_name}/EOLcause_test{threshold_name}.pkl', 'rb') as file:
    EOLcause= load(file)

performs = {}
thresholds = {}
for name in performs_info.keys():
    with open(f'./data/{dataset_name}/{name}_test{threshold_name}.pkl', 'rb') as file:
        performs[name] = load(file)
    with open(f'./data/{dataset_name}/{name}_test{threshold_name}_threshold.pkl', 'rb') as file:
        thresholds[name]= load(file)

EOLcause

['cumul time out',
 'cumul time out',
 'out of power',
 'out of power',
 'cumul time out',
 'out of power',
 'out of power',
 'out of power',
 'out of power',
 'cumul time out',
 'out of power',
 'out of power',
 'cumul time out',
 'out of power',
 'out of power',
 'out of power',
 'out of power',
 'out of power',
 'out of power',
 'out of power']

## Choose Library

In [36]:
library_address=f'./Library/MTL_2024-08-17_{dataset_name}/'

## Load Group Distribution

In [37]:
group_distributions={}
for perform_name in performs_info.keys():
    perform_info=performs_info[perform_name]

    perform_address=library_address+f'{perform_name}/'

    configs=torch.load(perform_address+'config.pth')
    model_state=torch.load(perform_address+'model_state.pth')
    model=GroupWaterCan(configs)
    model.load_state_dict(model_state)
    model.to(device)
    group_distributions[perform_name]=model
times_torch=[torch.tensor(time,device=device,dtype=torch.float32).unsqueeze(dim=1) for time in times]

# Compute and prepare A,b matrices and time

In [38]:
n_test_unit=len(times)
A,S=compute_eval_matrices(group_distributions,times_torch)
test_unit_list=range(n_test_unit)
b = performs

## Predicts and plot Performance Curves

In [39]:
n_train_unit =20 # 6
n_paths = 20    # 10 #
#gamma={'HI': 100}
gamma={'SOC': 100,
        'POS': 100,
        'CUM': 100}
ood_coef=1.

conserv_pred=0.5

In [40]:
config={'n_train_unit':n_train_unit,
        'n_paths':n_paths,
        'conserv_pred':conserv_pred,
        'ood_coef':ood_coef}

In [41]:
today = date.today()
threshold_text=0 if not threshold_name else threshold_name
exp=f'threshold{threshold_text}_{today}/'


#exp='try'

if exp == 'try':
    n_train_unit =5 #5#
    n_paths =5 #5# 
exp_address=library_address+f'Experiments/{exp}/'

max_life = 100
tol = 1e-4
max_iter = 100
import os
os.makedirs(exp_address, exist_ok=True)
with open(exp_address+'config.pth', "wb") as f:
    dump(config, f)

In [42]:
y_lim_={'SOC': (0.7,1),
        'POS':(0,6.1),
        'CUM':(0,28)}
loc_={'SOC': 'lower left',
        'POS':'upper left',
        'CUM':'upper left'}
        
monot_={'SOC':'-', #0.1
        'POS':'+', #0.1 
        'CUM':'+'}

gamma_={'SOC': 100, #0.001, #0.1
        'POS': 100, #0.1, #0.1 
        'CUM': 100} #1

time_est_={'SOC': None, #0.1
        'POS':None, #0.1 
        'CUM':None}

A_est_={'SOC': None, #0.1
        'POS':None, #0.1 
        'CUM':None}

b_est_={'SOC': None, #0.1
        'POS':None, #0.1 
        'CUM':None}

time_unit='hours'

In [43]:
# y_lim_={'HI': (0,1)}
# loc_={'HI': 'lower left'}
# monot_={'HI':'-'}
# gamma_={'HI': 100} #1
# time_est_={'HI': None}

# A_est_={'HI': None}

# b_est_={'HI': None}

# time_unit='cycle'


In [44]:
num_unit=len(times)
num_unit

20

In [45]:
long_life=max([len(time) for time in times])
pred_names='acc','csv','pred','unc'
end_time=int(long_life+5)
density_time=int((long_life+5)*2.5)
t = np.linspace(0, end_time, density_time)

num_unit=len(times)# RUL.shape[0]
if exp=='try': num_unit=1
accel=15*3600
trues = []
RMSEs={key: [] for key in pred_names}
predss={key: [] for key in pred_names}
causess={key: [] for key in pred_names}
for unit in range(num_unit): #[5]
        group_distribution_={}
        A_={}
        S_={}
        b_={}
        threshold_={}
        for name in performs_info.keys():
                group_distribution_[name]=group_distributions[name].get_sub_watercan(range(n_train_unit))
                A_[name]= A[name][unit][:,:n_train_unit]
                S_[name]= S[name][unit][:,:n_train_unit]
                b_[name]= b[name][unit]

                threshold=np.interp(t, times_thresholds[unit], thresholds[name][unit])
                threshold_[name]=torch.tensor(threshold,dtype=torch.float32,device=device)

        perform_info={'monot':monot_,
                        'threshold': threshold_,
                        'group_networks':group_distribution_,
                        'A': A_,
                        'b' : b_,
                        'S' : S_,
                        'gamma': gamma_,
                        'y_lim': y_lim_,
                        'loc' : loc_,
                        'time_est':time_est_,
                        'A_est':A_est_,
                        'b_est':b_est_,
        }
        preds,causes=make_rul_video(unit,t,times[unit],rul[unit],EOLcause[unit],**perform_info, time_unit=time_unit,
                                n_train_unit=n_train_unit, n_paths=20, ood_coef=ood_coef,
                                max_life = max_life,tol = tol,max_iter = max_iter,
                                accel=accel//3600, save=exp_address,conserv=conserv_pred)
        y_true=rul[unit]

        for name in pred_names:
                RMSE=np.sqrt(np.mean((preds[name]-y_true)**2))
                print(f"RMSE_{name}: {RMSE}")
                RMSEs[name].append(RMSE)
                predss[name].append(preds[name])
                causess[name].append(causes[name])
        trues.append(y_true)

torch.save(predss,f'{exp_address}'+'preds.pt')
torch.save(causess,f'{exp_address}'+'causes.pt')
torch.save(trues,f'{exp_address}'+'trues.pt')
print('----------------------------------------------------------------')
for name in pred_names:
        print(f"RMSE_{name} mean: {np.mean(RMSEs[name],axis=0)}")
        print(f"RMSE_{name} std: {np.std(RMSEs[name],axis=0)}")
print('----------------------------------------------------------------')

Moviepy - Building video ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit0_pred.mp4.
Moviepy - Writing video ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit0_pred.mp4



Moviepy - Done !
Moviepy - video ready ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit0_pred.mp4
RMSE_acc: 3.5715682517052896
RMSE_csv: 8.336861459711862
RMSE_pred: 6.360698684898448
RMSE_unc: 33.97562528213014
Moviepy - Building video ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit1_pred.mp4.
Moviepy - Writing video ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit1_pred.mp4



Moviepy - Done !
Moviepy - video ready ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit1_pred.mp4
RMSE_acc: 1.8033912049261662
RMSE_csv: 6.10462751899454
RMSE_pred: 4.144839759546991
RMSE_unc: 28.095052019449128
Moviepy - Building video ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit2_pred.mp4.
Moviepy - Writing video ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit2_pred.mp4



Moviepy - Done !
Moviepy - video ready ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit2_pred.mp4
RMSE_acc: 9.800846267734414
RMSE_csv: 14.789790173949136
RMSE_pred: 12.786968276783718
RMSE_unc: 16.76167782679253
Moviepy - Building video ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit3_pred.mp4.
Moviepy - Writing video ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit3_pred.mp4



Moviepy - Done !
Moviepy - video ready ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit3_pred.mp4
RMSE_acc: 6.181635039493803
RMSE_csv: 11.209670238393038
RMSE_pred: 9.17879597170502
RMSE_unc: 19.951978181614894
Moviepy - Building video ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit4_pred.mp4.
Moviepy - Writing video ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit4_pred.mp4



Moviepy - Done !
Moviepy - video ready ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit4_pred.mp4
RMSE_acc: 4.182590143572881
RMSE_csv: 8.755013994648602
RMSE_pred: 6.959780608061313
RMSE_unc: 20.834575575581933
Moviepy - Building video ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit5_pred.mp4.
Moviepy - Writing video ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit5_pred.mp4



Moviepy - Done !
Moviepy - video ready ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit5_pred.mp4
RMSE_acc: 4.257736709230391
RMSE_csv: 9.569722325274858
RMSE_pred: 7.2835896418717105
RMSE_unc: 19.250250033362974
Moviepy - Building video ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit6_pred.mp4.
Moviepy - Writing video ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit6_pred.mp4



Moviepy - Done !
Moviepy - video ready ./Library/MTL_2024-08-17_uav_data/Experiments/threshold0_2024-08-20//unit6_pred.mp4
RMSE_acc: 11.79484124850569
RMSE_csv: 17.26262501235685
RMSE_pred: 15.074452333426738
RMSE_unc: 14.16897912239424


In [ ]:
pred_names='acc','csv','pred','unc'

predss=torch.load(f'{exp_address}/preds.pt')
trues=torch.load(f'{exp_address}/trues.pt')

In [ ]:
pred_names='acc','csv','pred','unc'

predss=torch.load(f'{exp_address}/preds.pt')
trues=torch.load(f'{exp_address}/trues.pt')

In [ ]:
for name in pred_names:
        print(f'------------------------------- {name} RMSE -------------------------------')
        RMSEs=[]
        for unit in range(len(trues)):
                RMSE=np.sqrt(np.mean((predss[name][unit]-trues[unit])**2))
                print(f"unit {unit}: {RMSE}")
                RMSEs.append(RMSE)
        print('----------------------------------')
        print(f"RMSE_{name} mean: {np.mean(RMSEs,axis=0)}")
        print(f"RMSE_{name} std: {np.std(RMSEs,axis=0)}")
        print('----------------------------------')

------------------------------- acc RMSE -------------------------------
unit 0: 8.215497322801998
----------------------------------
RMSE_acc mean: 8.215497322801998
RMSE_acc std: 0.0
----------------------------------
------------------------------- csv RMSE -------------------------------
unit 0: 2.5107059841633816
----------------------------------
RMSE_csv mean: 2.5107059841633816
RMSE_csv std: 0.0
----------------------------------
------------------------------- pred RMSE -------------------------------
unit 0: 2.1250098156963246
----------------------------------
RMSE_pred mean: 2.1250098156963246
RMSE_pred std: 0.0
----------------------------------
------------------------------- unc RMSE -------------------------------
unit 0: 35.92247579574413
----------------------------------
RMSE_unc mean: 35.92247579574413
RMSE_unc std: 0.0
----------------------------------


In [ ]:
exp_address

'./Library/MTL_2024-08-17_uav_data/Experiments/try/'